In [3]:
#from __future__ import unicode_literals
import twitter_credentials as cred
import tweepy
import pickle
import os
import datetime
import time
import re
import boto

altered code from https://github.com/janvanzeghbroeck/urban-emoji/blob/master/twitter_api.py

In [ ]:
auth = tweepy.OAuthHandler(cred.consumer_key, cred.consumer_secret)
auth.set_access_token(cred.access_token, cred.access_token_secret)

api = tweepy.API(auth)



def get_tweets(topic, save_file_name, num_batches=25, num_tweets = 20, to_bucket = False): # num_batches * 100 is total tweets target
    tweets = set()
    # public_tweets = api.home_timeline()
    for i in range(num_batches):
        try:
            print('Loading', i+1, 'of', num_batches)
            for tweet in tweepy.Cursor(api.search, q=topic).items(num_tweets): #100 batches of 20

                if tweet.lang == 'en':
                    tweets.add(tweet.text) 

            time.sleep(35) 
        except:
            print('Waiting for API to allow more calls...')
            time.sleep(60)
            pass

    # if to_bucket:
        pass
    else:
        pickle.dump( tweets, open( "{}.pkl".format(save_file_name), "wb" ) )
        print('Succesfully pickled', len(tweets), 'tweets!')



if __name__ == '__main__':
    now = datetime.datetime.today().ctime()
    now = re.sub(' ','_',now)
    now = re.sub(':','-',now)


    # use boto to connect to aws buckets
    conn = boto.connect_s3(cred.aws_access_key, cred.aws_access_secret_key)

    # what bucket?
    bucket_name = 'emoji-tweets'

    # check if bucket exists if not make it
    if conn.lookup(bucket_name) is None:
        b = conn.create_bucket(bucket_name)
    else:
        b = conn.get_bucket(bucket_name)

    simple_words = ['is', 'it', 'the', 'are','vegan','✨', 'greta', 'ocasio', 'climate']
    
    for word in simple_words:
        pkl_name = './tweet_data/tweets_{}_{}'.format(now,word)
        s3_name = 'tweets_{}_{}.pkl'.format(now,word)
        loc_name = './tweet_data/tweets_{}_{}.pkl'.format(now,word)
        get_tweets(word, pkl_name, num_batches = 30, num_tweets = 100)

        # save the pkl file
        file_object = b.new_key(s3_name)#where to save
        file_object.set_contents_from_filename(loc_name)

        print('Successfully saved {} to S3 bucket {}'.format(s3_name,bucket_name))

    
    # to read the file
    #fil_object.get_contents_to_file('folder/file')

Loading 1 of 30
Loading 2 of 30
Loading 3 of 30
Loading 4 of 30
Loading 5 of 30
Loading 6 of 30
Loading 7 of 30
Loading 8 of 30
Loading 9 of 30
Loading 10 of 30
Loading 11 of 30
Loading 12 of 30
Loading 13 of 30
Loading 14 of 30
Loading 15 of 30
Loading 16 of 30
Loading 17 of 30
Loading 18 of 30
Loading 19 of 30
Loading 20 of 30
Loading 21 of 30
Waiting for API to allow more calls...
Loading 22 of 30
Waiting for API to allow more calls...
Loading 23 of 30
Waiting for API to allow more calls...
Loading 24 of 30
Loading 25 of 30
Loading 26 of 30
Loading 27 of 30
Loading 28 of 30
Loading 29 of 30
Loading 30 of 30
Succesfully pickled 2343 tweets!
Successfully saved tweets_Fri_Jun_28_09-31-44_2019_is.pkl to S3 bucket emoji-tweets
Loading 1 of 30
Loading 2 of 30
Loading 3 of 30
Loading 4 of 30
Loading 5 of 30
Loading 6 of 30
Loading 7 of 30
Loading 8 of 30
Loading 9 of 30
Loading 10 of 30
Loading 11 of 30
Loading 12 of 30
Loading 13 of 30
Loading 14 of 30
Waiting for API to allow more calls.

In [1]:
from os import listdir 
from os.path import isfile, join 
import pandas as pd
mypath = './tweet_data'
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [4]:
tweets = []
for i in files:
    file = './tweet_data/'+ i
    tweets += list(pickle.load(open(file,'rb')))

In [5]:
# read this many tweets!
len(tweets)

20732

In [7]:
# ----- creates the emoji df and adds the unichar
def df_emojis():
    # create full df_emoji
    df = pd.read_pickle('./data/df_emoji.pkl')
    return df

# ------------ finding the tweets with emojis
def yay_no(tweets,df_emojis):
    no_moji = []
    yay_moji = []
    for tweet in tweets:
        tweet = str(tweet) #some have type tweepy.models.Status
        yay = False
        for uni in df_emojis['unichar']:
            #if emoji in str(tweet):
            if uni in tweet:
                yay = True
        if yay:
            yay_moji.append(tweet)
        else: # else statement to create no_moji list
            no_moji.append(tweet)
    return yay_moji, no_moji


if __name__ == '__main__':

    tweets = list(set(tweets))

    df_emojis = df_emojis()

    yay_moji, no_moji = yay_no(tweets,df_emojis)

    pickle.dump( yay_moji, open( "./data/yay_moji.pkl", "wb"))
    pickle.dump( df_emojis, open( "./data/df_emojis.pkl", "wb"))

    print('Succesfully pickled {} tweets and emoji data frame'.format(len(yay_moji)))


Succesfully pickled 3909 tweets and emoji data frame


In [ ]:
len(list(set(tweets)))

In [12]:
len(no_moji) + len(yay_moji)

19702